# Onderzoek Data Science: Films
----

Voor de casusopdracht van het vak Data Science moet er een pipeline worden gemaakt, waarbij een toegewezen dataset wordt geanalyseerd. Dit data onderzoek is gebaseerd op de dataset over films en is opgezet door groep 6 van klas V2C. De dataset is toegewezen door de Hogeschool Utrecht en bevat onder andere:
- filmgegevens, waaronder duur, genres, taal, land van herkomst, budget en opbrengst;
- likes op facebook voor regisseur, hoofdrolspelers, totale cast en de film zelf;
- score op IMDB en aantal reviews. 


#### Hoofdvraag
Voor dit onderzoek is er een verplichte onderzoeksvraag, die luidt als volgt: 
- In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?

#### Onderzoeksvragen
Dit onderzoek bevat een aantal onderzoeksvragen die wij zullen behandelen:

|Beeordeling                    |Onderzoeksvraag                                                                       |
|:------------------------------|:-------------------                                                                  |
|Externe dataset                |Hoeveel effect heeft de lengte van een trailer op de omzet van de film?               |
|Correlatieonderzoek            |Is een film succesvoller op basis van het aantal likes die een acteur (of acteurs) heeft/hebben op Facebook?                                                                                              | 
|Supervised machine learning    |In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?                                                                                                          |
|Unsupervised machine learning  |Is een film succesvoller op basis van het aantal likes die een acteur (of acteurs) heeft/hebben op Facebook?                                                                                              |
|Interactieve visualisatie      |Wat is de totale winst van alle films per jaar?                                       |

#### Z-test
Voor de hypothesetoets Z-test moet het volgende gebeuren.	
Een filmcriticus stelt dat de score van Engelstalige films lager is dan gemiddeld.
Wij moeten onderzoeken met de dataset of deze filmcriticus gelijk heeft. We nemen een steekproef (met pandas.DataFrame.sample(n=100,random_state=1)) van 100 Engelstalige films en beschouwen de hele dataset als populatie. Ook nemen we als betrouwbaarheid 90%. We gebruiken van de dataset alleen de filmgegevens waarbij zowel de taal (language) als de score (imdb_score) bekend zijn.

#### Teamleden
Het team bestaat uit de volgende drie personen:
- Sebastiaan Jansen
- Skott de Koster
- Mustafa Toprak

### Inhoudsopgave
---
1. Data Collection
2. Data Processing
3. Data Cleaning
4. Data Exploration & Analysis
5. Model building
6. Visualization
7. Communication

## Stap 1: Data collection
---

De dataset is aan ons gegeven door de Hogeschool Utrecht als onderdeel van de opdracht. Deze dataset houd een groot CSV bestand in met informatie over films. We hebben dus nu de data verzameld en gaan het nu inlezen.
De dataset is opgehaald van de volgende GitHub link: https://github.com/tijmenjoppe/ComputationalModelling-student/tree/master/casus/movie

Ook maken wij gebruik van een externe dataset. Deze externe dataset bevat een link ID naar de trailers van de films die op YouTube te vinden zijn. De externe dataset is opgehaald van de link: https://grouplens.org/datasets/movielens/20m-youtube/

## Stap 2: Data processing
---

We gaan nu de dataset inlezen en verwerken om het beter te kunnen bekijken. Om te beginnen importeren we de benodigde Python libraries:

In [ ]:
# Deze libraries zijn voor het verwerken van de data.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from scipy import stats

# Deze libraries zijn voor het verkrijgen van YouTube video gegevens. Dit is relevant voor het gebruik van de externe dataset.
import pafy
from youtube_dl import YoutubeDL
import datetime 

Vervolgens lezen we de eerste (toegewezen) dataset in. Hiervoor gebruiken we pandas.read_csv en slaan we dit op in de dataframe "films". 
Aangezien de dataset erg groot is, laten we voor nu alleen de eerste 5 rijen zien. Dit doen we door een variabele films.head() aan te roepen:

In [ ]:
films = pd.read_csv("movie.csv", sep=",")
films.head()

Hetzelfde doen we met de tweede (externe) dataset, deze noemen we "films_extern": 

In [ ]:
films_extern = pd.read_csv("ml-youtube.csv", sep=",")
films_extern.head()

We zien dus dat de eerste dataset uit veel films bestaat. De tweede dataset heeft er nog veel meer, maar niet alle data in de sets is bruikbaar, dus dat gaan we nu schoonmaken.

# Stap 3: Data cleaning
---

We gaan nu de data schoon maken. Om te beginnen schonen we beide datasets op door alle data die niet relevant is te verwijderen:

In [ ]:
to_drop = ["color", "facenumber_in_poster", "country", "aspect_ratio", "content_rating"]
films.drop(to_drop, inplace=True, axis=1)

films_extern.drop("movieId", inplace=True, axis=1)

In [ ]:
# We verwijderen de rijen met NaN gegevens in beide datasets, aangezien dit data is die we niet kunnen gebruiken.
films = films.dropna()
films_extern = films_extern.dropna()

In [ ]:
# We halen alle duplicates eruit.
films = films.drop_duplicates(subset="movie_title")
films_extern = films_extern.drop_duplicates(subset="title")

In [ ]:
# We halen een whitespace weg in de eerste dataset die er niet in hoort te zitten.
# We slicen de laatste 7 characters van de title column (het jaar dat de film uitkwam) in de tweede dataset weg,
# zodat de column overeen komt met de column in de eerste dataset.
films.update(films["movie_title"].str[:-1])
films_extern.update(films_extern["title"].str[:-7])

In [ ]:
# We setten de indexes van beide datasets naar de titel van de films.
films.set_index("movie_title", inplace=True)
films_extern.set_index("title", inplace=True)

In [ ]:
# We sorteren beide datasets zodat de waardes gelijk zullen zijn.
films = films.sort_index()
films_extern = films_extern.sort_index()

Nu ziet de eerste dataset er zo uit:

In [ ]:
films.head()

En ziet de tweede dataset er zo uit:

In [ ]:
films_extern.head()

Nu gaan we beide datasets samenvoegen tot 1 dataframe, deze noemen we 'films_extern_merge'. Het idee is om de youtubeId column van de tweede dataset te verwerken in de eerste dataset. Deze gemergde dataset gaan we later gebruiken.

In [ ]:
# Omdat de youtubeId column het enige is wat relevant is, is dat ook het enige veld dat we toevoegen. 
# De indexes van de eerste en tweede dataset worden samengevoegd tot 1 index genaamd title.
films_extern_merge = pd.merge(films, films_extern, left_index=True, right_index=True)
films_extern_merge.index.name = "title"

# Tot slot droppen we de niet-relevante kollomen. Om code te besparen, overschrijven we alleen de dataframe met de relevante kollomen.
films_extern_merge = films_extern_merge[["gross", "youtubeId"]]
films_extern_merge

Je kunt zien dat er 2827 films zijn. Dit is bijna de helft minder dan er in de oorspronkelijke eerste dataset stond, dit is omdat we de onbruikbare data hebben verwijderd en er alleen bruikbare data is overgebleven.
We gaan de gemergede dataset nu verkennen en analyseren.

# Stap 4: Data exploration & analysis
---

Om de data te verkennen en er een betere grip op te krijgen zullen wij een paar commando's loslaten op de verwerkte data. Als eerste zullen wij een .describe gebruiken om een overzicht te krijgen van alle info van de numerieke waardes. Op deze manier kunnen wij bijvoorbeeld de gemiddelde lengte van een film zien. Dit is bij deze dataset dus 105 minuten.

In [ ]:
films.describe()

Om de data verder te verkennen en een beter gevoel te krijgen kunnen wij gebruik maken van grafieken. Zo staat hieronder bijvoorbeeld een staafdiagram van het aantal films per jaartal. In de x as staat het jaartal, deze zijn gesorteerd bij hoeveel films er in dat jaartal zijn uitgekomen die in deze dataset staan. De meeste films uit deze dataset komen dus uit 2002.

In [ ]:
films["title_year"].value_counts().plot(kind="bar", width=0.5,figsize=(20,5));
plt.xlabel("Jaartal", labelpad=14)
plt.ylabel("Aantal films", labelpad=14)
plt.title("Aantal films per jaartal", y=1.02);

## Onderzoeksvragen
---

Voor deze notebook moeten er verschillende onderzoeksvragen worden uitgewerkt, elk met hun eigen antwoord en andere manier van oplossen. Deze zullen nu één voor één worden uitgewerkt.

### Vraag 1: Hoeveel effect heeft de lengte van een trailer op de omzet van de film?
---

Trailers zijn een belangrijk onderdeel van films. Je wilt immers wel enigzins weten wat je kan verwachten van de film.<br>
Dit kan ook een factor spelen in de omzet van de film. Want als de trailer slecht is zal je waarschijnlijk ook niet snel naar de film gaan, en dan zal de film dus ook minder omzet hebben.<br>
We gebruiken hier alleen de lengte van de trailer, om het simpel te houden.

Voor deze vraag maken we gebruik van de gemergde dataset uit stap 3. Zoals je misschien al hebt kunnen raden bestaat de externe dataset uit ID's voor YouTube video's. Deze verwijzen naar de trailers van de films van de toegewezen dataset. We hebben bij stap 3 de toegewezen en externe dataset gemerged tot 1 dataset, deze gaan we nu gebruiken.

In [ ]:
films_extern_merge.iloc[:,-1].head()

Hier komt de pafy library ter zake. Om een voorbeeld te geven over wat het kan doen maken we eerst een video variabele aan:

In [ ]:
# Om het simpel te houden gebruiken we voor nu maar de youtubeId van de eerste film in de dataframe.
url = films_extern_merge.iloc[0, -1]
video = pafy.new(url)

Van deze video kunnen we nu verschillende data ophalen, hier een paar voorbeelden:

In [ ]:
# De titel van de video
print("De titel van de video is: \n" + str(video.title))
print("\n")

# De lengte van de video in secondes
print("De lengte van de video in secondes is: \n" + str(video.length))
print("\n")

# De concrete duratie van de video
print("De concrete duratie van de video is: \n" + str(video.duration))

Er zijn nog veel andere statistieken die kunnen worden opgehaad met behulp van pafy, maar hiervoor is een YouTube API account voor nodig en is ook niet voor ons relevant. We focussen ons op de <b>concrete duratie</b> van de video.

Dus, heeft de duratie van de trailer invloed op de omzet van de film? Om daar achter te komen maken we nieuwe dataframe aan, deze pakt 100 willekeurige films uit de dataset:

In [ ]:
# De dataframe heet "random_films".
random_films = films_extern_merge.sample(n = 100)
# In deze list slaan we zometeen de duraties op.
duratie = []
# We loopen door random_films heen en extracten van elke video de duratie in secondes.
# LET OP! Dit duurt een tijdje, aangezien er elke keer opnieuw een HTTP request wordt gemaakt. Je kunt ook de errors en warnings negeren.
for x in random_films["youtubeId"].values:
    try:
        # We appenden de lengte van de trailer toe aan de list.
        duratie.append(pafy.new(x).length)
    except Exception:
        # Sommige video's zijn niet meer te zien op YouTube, hier kan de data dus ook niet van worden opgehaald.
        # Maar aangezien de waardes niet door elkaar mogen lopen, appenden we een Null waarde.
        duratie.append(None)
        continue

# We voegen een kolom toe met de waardes van duratie.
random_films["trailer_length"] = duratie
# We halen alle trailers eruit met een duratie van meer dan 400 seconden, aangezien deze vaak niet kloppen.
random_films = random_films[random_films.trailer_length < 400]
random_films

Nu maken we een mooie scatterplot om te zien of er een verband is tussen de omzet en trailer duratie:

In [ ]:
sns.regplot(x=random_films["trailer_length"], y=random_films["gross"])

Elke uitkomst die je krijgt zal een ander resultaat geven. Maar wat wel duidelijk is in elke uitkomst is dat er geen specifiek partroon te zien is.
<br>
**Conclusie:** We kunnen dus concluderen dat er <b>geen</b> duidelijk verband is tussen de lengte van de trailer en de omzet van de film.

### Vraag 2: Is een film succesvoller op basis van het aantal likes die een acteur (of acteurs) heeft/hebben op Facebook?
---

Social media speelt tot op de dag van vandaag een hele belangrijke rol in ons dagelijks leven. Overal zien we beroemdheden en sommige mensen willen heel graag op deze beroemdheden lijken. Hierdoor wordt het gedrag van een individu bepaald. 

Tegenwoordig draaien de meeste dingen om likes en dat geldt ook voor deze deelvraag. Is het echt waar dat een film succesvoller is op basis van het aantal likes die een acteur (of acteurs) heeft/hebben op Facebook?

Om die vraag te kunnen beantwoorden, moeten we eerst twee beoordelingscriteria langs. Het eerste criterium zal gaan over het correlatieonderzoek. Bij het tweede criterium wordt er gebruik gemaakt van een **unsupervised machine learning** techniek.

#### Correlatieonderzoek

Met behulp van correlatie wordt de samenhang gemeten tussen twee variabelen. 

Het is belangrijk dat je de waarde van een andere variabele goed kunt voorspellen o.b.v. waardes van een variabele die je al kent. Hoe groter de correlatie, des te hoger de voorspellingskracht is.<br>
Om deze vraag te kunnen beantwoorden, worden de variabelen IMDB-score en het gemiddelde aantal likes van de acteurs genomen.</br>

Hieronder maken we eerst een extra kolom aan die het gemiddelde neemt van het aantal likes van de acteurs. Deze kolom voegen we toe aan de dataset films:

In [ ]:
# Voeg een kolom van het gemiddeld aantal likes van de acteurs toe aan de dataset films.
films["mean_likes_actors"] = (films["actor_1_facebook_likes"] + 
                              films["actor_2_facebook_likes"] + 
                              films["actor_3_facebook_likes"]) / 3
films

Zoals gezegd hebben de worden de variabelen IMDB-score en mean_likes_actors gebruikt. Hieronder wordt een scatter-plot gemaakt van deze variabelen:

In [ ]:
# Maak gebruik van de Seaborn style.
plt.style.use("seaborn")

# Maak een scatter-plot van de variabelen.
plt.scatter(films.imdb_score, films.mean_likes_actors, 
            s=40, 
            c="red", edgecolor="black", 
            linewidth=0.75, alpha=0.75)

# Voeg kenmerken toe aan de plot.
plt.xlabel("IMDB-score")
plt.ylabel("Aantal likes")
plt.title("Correlatie tussen IMDB-score en aantal likes")

Zoals te zien is liggen de punten heel dicht op elkaar. Nu laten we nog even de r-waardes (richtingscoëfficient) zien van de correlatie tussen deze twee variabelen. Hiervoor maken we een aparte dataset aan:

In [ ]:
# Voeg de kolommen imdb_score en mean_likes_actors toe aan een nieuwe dataset.
films_correlation = films[["imdb_score", "mean_likes_actors"]]
films_correlation

In [ ]:
# Toon de correlatie tussen de twee variabelen.
films_correlation.corr().style.background_gradient(cmap="coolwarm")

*We hebben een aparte dataset aangemaakt voor deze correlatie, omdat de andere gegevens niet relevant zijn voor het beantwoorden van de vraag. Mocht u geïntereseerd zijn in de correlatie met alle variabelen, kunt u het onderstaande statement runnen:*

In [ ]:
# Geef een overzicht van de correlatie met alle variabelen.
# films.corr()

Voor een goede correlatie is het belangrijk dat de richtingscoëfficient dicht bij de 1 zit ($ -1 ≤ r ≤ 1$). We zien uit de bovenstaande gegevens dat de richtingscoëfficient voor imdb_score en mean_likes_actors 0.187995 is.

#### Conclusie

Er is sprake van een positieve correlatie, maar er is **geen** sprake van een sterk verband tussen het succes van een film en het aantal likes van acteurs.

#### Unsupervised machine learning

### Vraag 3: In hoeverre is de omzet van een film te voorspellen op basis van de populariteit op Facebook en IMDB zelf?
---

Omzet is erg belangrijk voor een film. Het is immers waar een film voor wordt gemaakt: om omzet te draaien. Deze omzet komt natuurlijk van alle mensen die deze film hebben gezien in de bioscoop en uiteindelijk ook de DVD's. <br>
Maar natuurlijk gaat de gemiddelde persoon niet naar elke film die ooit uitkomt. Als een film een lage score heeft op IMDB of weinig Facebook likes heeft, geeft dat al gauw een signaal dat de film niet zo goed is. Het is dus veilig om te zeggen dat zo'n score invloed heeft op de populariteit van een film. <br>
Maar heeft dit ook veel effect op de omzet van de film? En tot hoeverre kunnen we hiermee de omzet van toekomstige films voorspellen? Dat gaan we nu uitzoeken.

Voor deze vraag wordt gebruikt gemaakt van <b>supervised machine learning.</b> Dit is gebaseerd op een dataset waarvan de uitkomst al bekend is en houdt in dat we de machine letterlijk "begeleiden" in zijn taak om iets uit te voeren. We geven de machine data om te onderzoeken en zorgen dat we deze een goed antwoord kan geven.<br>
Onder supervised machine learning worden er binnen het vak Data Science twee technieken verstaan:
    
|Techniek                       |Omschrijving                          |                                
|:------------------------------|:-------------------------------------|
|Lineaire regressie             |Lineaire regressie probeert de relatie tussen twee variabelen te modelleren door een <b>lineaire vergelijking</b> aan te passen aan de waargenomen gegevens.|
|Classificatie                  |Classificatie is een geordende reeks gerelateerde categorieën die wordt gebruikt om gegevens te <b>groeperen op basis van overeenkomsten</b>. Binnen het vak Data Science bevat het 2 onderdelen: <b>decision trees</b> en <b>K-nearest neighbors</b>.                                            |

Om een betrouwbaar onderzoek uit te voeren maken we gebruik van <b>beide</b> technieken. We beginnen met lineaire regressie, dan classificatie. Maar voordat we beginnen moeten we een paar dingen instellen:

In [ ]:
# We maken een kopie van de films dataset met alleen de relevante data.
films_sml = films[["imdb_score", "movie_facebook_likes", "gross"]].copy()

# We stellen de variabele X op, deze vertegenwoordigd de feature matrix (X-as).
# Het bevat de totale filmscore op IMDB en de het totale aantal facebook likes per film.
feature_cols = ["imdb_score","movie_facebook_likes"]

X = films_sml[feature_cols]

# We stellen we de variabele Y op, deze vertegenwoordigd de target vector(Y-as).
# Het bevat de totale omzet per film.
Y = films_sml.gross

# We splitten de data op in test en training sets. 75% van de data gaat naar de training set, 25% gaat naar de test set.
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

# We printen de shape van de training sets.
print("De shape van de X train set is " + str(X_train.shape))
print("De shape van de Y train set is " + str(Y_train.shape))
print("De shape van de X test set is " + str(X_test.shape))
print("De shape van de Y test set is " + str(Y_test.shape))
print("\n")

In principe hebben we nu de eerste paar stappen van de werkwijze van Scikit-learn toegespast. De variabelen kunnen we hergebruiken in beide technieken. Zo hoeven we ze niet telkens opnieuw aan te maken.<br>
Nu we de voorbereidingen hebben gemaakt, kunnen we beginnen met lineaire regressie.

In [ ]:
# We importeren het benodigde onderdeel van scikit-learn.
from sklearn.linear_model import LinearRegression

# We instantieeren de lineaire regressie.
linreg = LinearRegression()

# We fitten het model aan de training data, oftewel we leren de coeffienten.
linreg.fit(X_train, Y_train)

# We printen the onderschepping en coëfficiënten.
print("De onderschepping van de lineaire regressie is " + str(linreg.intercept_))
print("De coëfficiënt van de lineaire regressie is " + str(linreg.coef_))
print("\n")

# We paren de coëfficiënten met de feature namen.
list(zip(feature_cols, linreg.coef_))

# We maken voorspellingen op de test set.
Y_pred = linreg.predict(X_test)

# Tot slot berekenen we de RMSE (Root Mean Squared Error) voor de omzet voorspellingen.
print("De RMSE van de omzet voorspellingen is " + str(np.sqrt(metrics.mean_squared_error(Y_test, Y_pred))))

Deze nummers zeggen niet veel duidelijks. We willen de RMSE namelijk zo laag mogelijk hebben. Dit kunnen we proberen om waar te maken door de code opnieuw te draaien, maar met 1 feature in plaats van 2:

Er is hier een mogelijk verklaring voor: <b>correlatie.</b> Je hebt immers voor lineaire regressie enige <b>correlatie nodig</b>. Om te checken of er correlatie is doen we het volgende:

In [ ]:
films_sml.corr()

We zien dat de correlatie tussen de feature matrix en target vector erg klein is. Voor het correct uitvoeren van lineaire regressie is een minimale correlatie nodig van <b>0.7</b> (positief) of <b>-0.7</b> (negatief).

Als we nu een aantal scatterplots opstellen tussen de feature matrix en target vector kunnen we zien waarom er weinig correlatie is:

In [ ]:
sns.pairplot(films_sml, x_vars=["imdb_score","movie_facebook_likes"], y_vars="gross", height=7, aspect=0.9, kind="reg")

In beide scatterplots is geen goede correlatie te zien. We kunnen nu dus concluderen dat er met lineaire regressie <b>geen</b> goede voorspelling kan worden gemaakt. <br>

Maar hoe zit het met classificatie? Dat gaan we nu uitzoeken.

Zoals eerder vermeld vallen er binnen het vak Data Science 2 technieken onder classificatie: **Decision Trees** en **k-Nearest Neighbours**. Binnen deze context kunnen we alleen gebruik maken van Decision Trees. De techniek k-Nearest Neighbours is namelijk voor het indelen van een **nieuw object**, wat in principe niet kan bijdragen aan het beantwoorden van de vraag.

Decision trees is een algoritme dat telkens een vraag bedenkt om de bestaande objecten in te delen in groepen, totdat deze uiteindelijk nauwkeurig zijn ingedeeld. Er wordt mee voorspeld wat het type is van verschillende cultivars, bijvoorbeeld een appel als je fruit gaat onderzoeken.<br>
In dit geval zal het proberen om de types IMDB score of facebook likes te vinden.<br>
Laten we beginnen.

In [ ]:
# We importeren het benodigde onderdeel van scikit-learn.
from sklearn.tree import DecisionTreeClassifier

# We instantieeren de decision tree classifier.
clf = DecisionTreeClassifier()

# We fitten de decision tree op de training sets.
clf = clf.fit(X_train, Y_train)

# We voorspellen hoe nauwkeurig het algoritme de type van de cultivars kan voorspellen.
Y_pred = clf.predict(X_test)
print("Nauwkeurigheid:", metrics.accuracy_score(Y_test, Y_pred))

Er is een 0.0 in nauwkeurigheid. Dit is heel slecht, want dit laat zien dat het algoritme de types absoluut niet kan voorspellen. Dit kunnen we vaststellen door Graphviz te gebruiken:

In [ ]:
# Dit is een fix voor een error met graphviz.
# Het zorgt ervoor dat de error "InvocationException: GraphViz's executables not found" niet voorkomt.
import os
os.environ["PATH"] = os.environ["PATH"] + ";" + os.environ["CONDA_PREFIX"] + r"\Library\bin\graphviz"

# We importeren de benodigde libraries.
from sklearn.datasets import load_iris
from sklearn import tree
from graphviz import Source
from IPython.display import SVG

# We genereren een graph met de decision tree en visualeren die.
graph = Source(tree.export_graphviz(clf, out_file=None, feature_names=X.columns))
SVG(graph.pipe(format="svg"))

Deze graphic is absoluut veel te groot en je wordt er niet wijzer uit. We kunnen dus concluderen dat ook met decision trees de omzet **niet goed voorspeld kan worden.**

Wat kan de reden zijn dat de omzet niet goed voorspeld kan worden? Als we nog een keer naar de scatterplots kijken...

In [ ]:
sns.pairplot(films_sml, x_vars=["imdb_score","movie_facebook_likes"], y_vars="gross", height=7, aspect=0.9, kind="reg")

Dan zien we dat imdb_score en movie_facebook_likes compleet andere eigenschappen hebben. De IMDB score is een getal van 1 tot 10, terwijl de facebook likes kunnen lopen tot in de 300.000. De 2 variabelen verschillen te veel van elkaar en hebben te weinig correlatie voor een goede voorspelling.

#### Conclusie

De omzet kan **niet** goed worden voorspeld met supervised machine learning, omdat de variabelen te veel van elkaar verschillen en er te weinig correlatie is.

# Stap 7. Communication
---

Als laatste stap volgt hier een beschrijving van enkele punten. 

Allereerst wordt er een beschrijving gegeven van hoe het bouwen van de Data Science-pipline is aangepakt. Daarna wordt er een beschrijving van de methode van dataverzamelingen en -bewerkingen gegeven. <br>Vervolgens wordt de dataopbouw, de analyses en de betekenis van de resultaten besproken. Er wordt concreet antwoord gegeven op de onderzoeksvragen en er volgt een beargumenteerde conclusie. Als laatst zijn er gebruikte bronnen te vinden.

## Aanpak Data Science-pipeline
---

## Methode dataverzameling en -bewerking
---

## Dataopbouw, analyses en resultaten
---

## Antwoorden onderzoeksvragen
---

## Conclusie
---

## Bronnen
---

Voor het samenvoegen van het gemiddelde van meerdere kolommen:<br> 
https://stackoverflow.com/questions/34734940/row-wise-average-for-a-subset-of-columns-with-missing-values<br>
Voor het correlatieonderzoek:<br>
https://canvas.hu.nl/courses/7546/files/folder/slides?preview=765874 & https://canvas.hu.nl/courses/7546/pages/lineaire-regressie?module_item_id=248049 <br>
Voor het stylen van de scatter-plot:<br> https://www.youtube.com/watch?v=zZZ_RCwp49g <br>
Voor de uitleg van lineaire regressie:<br> http://www.stat.yale.edu/Courses/1997-98/101/linreg.htm <br>
Voor de uitleg van classificatie:<br>
https://www.cso.ie/en/methods/classifications/classificationsexplained/   